In [29]:
import os

os.chdir("..")

from dotenv import load_dotenv

load_dotenv()

os.environ.setdefault(
    "USER_AGENT", "intellijupyter:llm-lab:0.1.0 (by github.com/kvdomingo)"
)

from pprint import PrettyPrinter

import ollama
from langchain_core.messages import HumanMessage
from langchain_core.tools import tool
from langchain_ollama import ChatOllama

from common.settings import settings

In [4]:
client = ollama.Client(host=str(settings.OLLAMA_URL))
chat_model_name = "llama3.2"
client.pull(chat_model_name)

{'status': 'success'}

In [6]:
@tool
def multiply(a: int | float, b: int | float) -> int | float:
    """Multiply two numbers."""
    return a * b


@tool
def add(a: int | float, b: int | float) -> int | float:
    """Add two numbers."""
    return a + b

In [14]:
llm = ChatOllama(base_url=str(settings.OLLAMA_URL), model=chat_model_name)
tools = [multiply, add]
tool_selection = {t.name: t for t in tools}
llm_tools = llm.bind_tools([multiply, add])

In [31]:
messages = [HumanMessage("What is 6 + 9? Also, what is 2 * 3?")]
res = llm_tools.invoke(messages)
PrettyPrinter(indent=2).pprint(res.tool_calls)

for tc in res.tool_calls:
    bot_response = tool_selection[tc["name"]].invoke(tc)
    print(bot_response.content)

[ { 'args': {'a': '6', 'b': '9'},
    'id': 'd4887564-b3df-4784-ac05-689f3788b697',
    'name': 'add',
    'type': 'tool_call'},
  { 'args': {'a': '2', 'b': '3'},
    'id': '4e278b7b-c784-45a4-b9b3-8ddc9a5e6c95',
    'name': 'multiply',
    'type': 'tool_call'}]
15
6
